# Assignment: Interactive Dashboards

In [2]:
#!pip install jupyter-dash

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 

import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
#import dash_core_components as dcc
from dash import dcc
#import dash_html_components as html
from dash import html
from dash.dependencies import Input, Output

## Diamonds Dataset

In [5]:
df = pd.read_csv('Diamonds Prices2022.csv',index_col=0)

df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [6]:
df.info()

df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53943 entries, 1 to 53943
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53943 non-null  float64
 1   cut      53943 non-null  object 
 2   color    53943 non-null  object 
 3   clarity  53943 non-null  object 
 4   depth    53943 non-null  float64
 5   table    53943 non-null  float64
 6   price    53943 non-null  int64  
 7   x        53943 non-null  float64
 8   y        53943 non-null  float64
 9   z        53943 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.5+ MB


(53943, 10)

In [7]:
df.describe()

,carat,depth,table,price,x,y,z
count,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000
mean,0.797935,61.749322,57.457251,3932.734294,5.731158,5.734526,3.538730
std,0.473999,1.432626,2.234549,3989.338447,1.121730,1.142103,0.705679
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.000000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [8]:
df['cut'].value_counts()

Ideal        21551
Premium      13793
Very Good    12083
Good          4906
Fair          1610
Name: cut, dtype: int64

In [9]:
df['clarity'].value_counts()

SI1     13067
VS2     12259
SI2      9194
VS1      8171
VVS2     5066
VVS1     3655
IF       1790
I1        741
Name: clarity, dtype: int64

In [10]:
df.describe(exclude='number')

,cut,color,clarity
count,53943,53943,53943
unique,5,7,8
top,Ideal,G,SI1
freq,21551,11292,13067


## Plotly Dash App

In [43]:
app = JupyterDash(__name__)

df = df.groupby(['cut', 'color', 'clarity'])[['carat','price']].mean()
df.reset_index(inplace=True)
df.head()

,cut,color,clarity,carat,price
0,Fair,D,I1,1.877500,7383.000000
1,Fair,D,IF,0.380000,1619.666667
2,Fair,D,SI1,0.913793,4273.344828
3,Fair,D,SI2,1.016964,4355.142857
4,Fair,D,VS1,0.630000,2921.200000


In [44]:
# App layout
app.layout = html.Div([

    html.H1("Diamonds Dashboard", style={'text-align': 'center'}),

    dcc.Tabs([
        dcc.Tab(label='Diamond Cuts', children=[
            html.H4("Pick Diamond Cut Type", style={'text-align': 'left'}),

            dcc.Dropdown(id="slct_cut",
                         # Note: make sure that the data type in the value matches
                         # the data type in the dataset
                         options=[
                             {"label": "Good", "value": 'Good'},
                             {"label": "Ideal", "value": 'Ideal'},
                             {"label": "Premium", "value": 'Premium'},
                             {"label": "Very Good", "value": 'Very Good'},
                             {"label": "Fair", "value": 'Fair'}
                         ],
                         multi=False,
                         value='Good',
                         style={'width': "25%"}
                         ),

            html.Br(),

            # Place holder --> currently empty 
            html.Div(id='output_container', children=[]),

            html.Br(),
            
            # Place holder --> currently empty 
            dcc.Graph(id='diamond_cut_scatter', figure={}),
            
            html.Br(),
            
            
        ]),
        
    #----------------------------------------------------------------------------------       
        dcc.Tab(label='Diamond Claritys', children=[
    
            html.Br(),
            html.Br(),
            html.Label('Pick Diamond Clarity'), # Add a Label

            dcc.RadioItems(id="slct_clarity",
                options=[
                    {'label': 'I1', 'value': 'I1'},
                    {'label': 'SI1', 'value': 'SI1'},
                    {'label': 'SI2', 'value': 'SI2'},
                    {'label': 'VS1', 'value': 'VS1'},
                    {'label': 'VS2', 'value': 'VS2'},
                    {'label': 'VVS1', 'value': 'VVS1'},
                    {'label': 'VVS2', 'value': 'VVS2'},
                    {'label': 'IF', 'value': 'IF'},
                ],
                value='I1'
            ),

            html.Br(),

            # Place holder --> currently empty 
            dcc.Graph(id='diamond_clarity_scatter', figure={}),

            html.Br(),
        ]),
        
    #----------------------------------------------------------------------------------        
        dcc.Tab(label='Diamond Prices', children=[
            html.H4("Choose Standerd", style={'text-align': 'left'}),

            dcc.RadioItems(id="slct_for_price",
                         # Note: make sure that the data type in the value matches
                         # the data type in the dataset
                         options=[
                             {"label": "plotly", "value": "plotly"},
                             {"label": "plotly_white", "value": "plotly_white"},
                             {"label": "plotly_dark", "value": "plotly_dark"},
                             {"label": "ggplot2", "value": "ggplot2"},
                             {"label": "seaborn", "value": "seaborn"},
                             {"label": "simple_white", "value": "simple_white"},
                             {"label": "none", "value": "none"}
                         ],
                         value='plotly',
                         inline=True
            ),
            

            html.Br(),
            html.Br(),
            
            # Place holder --> currently empty 
            dcc.Graph(id='diamond_cut_prices', figure={}),
            
            html.Br(),
            
            # Place holder --> currently empty 
            dcc.Graph(id='diamond_clarity_prices', figure={}),
            
            html.Br(),
            
        ]),
        
    ])
])
#---------------------------------------------------------------------------------- 

@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='diamond_cut_scatter', component_property='figure'),
     Output(component_id='diamond_clarity_scatter', component_property='figure'),
     Output(component_id='diamond_cut_prices', component_property='figure'),
     Output(component_id='diamond_clarity_prices', component_property='figure')
    ],

    [Input(component_id='slct_cut', component_property='value'),
     Input(component_id='slct_clarity', component_property='value'),
     Input(component_id='slct_for_price', component_property='value')],
    
)
#---------------------------------------------------------------------------------- 

def update_graph(option_slctd, option_clarity, option_slct_for_price):
    

    container = "The diamond cut type chosen by user was: {}".format(option_slctd)

    dff = df.copy()
    dff = dff[dff["cut"] == option_slctd]

    # Plotly Express
    fig = px.scatter(
        data_frame=dff, x='carat', y='price', color = 'clarity',
        template='plotly_dark' # setting parameter
    )
    
    fig.update_layout(
        title_text="Diamonds Prices ($) per Carat and Clarity",
        title_xanchor="center",
        title_font=dict(size=24),
        title_x=0.5,
    )
    
  #----------------------------------------------------------------------------------

    dff2 = df.copy()
    dff2 = dff2[dff2["clarity"] == option_clarity]
    
    # Plotly Express
    fig2 = px.scatter(
        data_frame=dff2, x='carat', y='price', color = 'cut',
        template='plotly_dark' # setting parameter
    )
    
    fig2.update_layout(
        title_text="Diamonds Prices ($) per Carat and Cut",
        title_xanchor="center",
        title_font=dict(size=24),
        title_x=0.5,
    )
    
    #----------------------------------------------------------------------------------
    dff3 = df.copy()


    # Plotly Express
    fig3 = px.bar(
        data_frame = dff3,
        x = dff3["cut"].value_counts().index, # Grab labels 
        y = dff3[["cut","price"]].groupby('cut').mean()['price'], # Grab values
        title = "Diamonds Average Prices per Cut",
        labels={"x":"Cut", "y":"Price ($)"}, # Re-label axis to appropriate names
        template= option_slct_for_price
    )

    fig3.update_layout(
        title_text="Diamonds Average Prices per Cut",
        title_xanchor="center",
        title_font=dict(size=24),
        title_x=0.5,
    )

    dff4 = df.copy()
    # Plotly Express
    fig4 = px.bar(
        data_frame = dff4,
        x = dff4["clarity"].value_counts().index, # Grab labels 
        y = dff4[["clarity","price"]].groupby('clarity').mean()['price'], # Grab values
        title = "Diamonds Average Prices per Clarity",
        labels={"x":"Clarity", "y":"Price ($)"}, # Re-label axis to appropriate names
        template= option_slct_for_price
    )

    fig4.update_layout(
        title_text="Diamonds Average Prices per Clarity",
        title_xanchor="center",
        title_font=dict(size=24),
        title_x=0.5,
    )

        
        
    return container, fig, fig2, fig3, fig4

# ------------------------------------------------------------------------------

if __name__ == '__main__':
    app.run_server(debug=False, host="localhost",port=8052)

 * Running on http://localhost:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Aug/2022 17:27:37] "GET /_alive_f7489dbb-836a-464f-a88c-ae6ecf59a8b0 HTTP/1.1" 200 -


Dash app running on http://localhost:8052/


127.0.0.1 - - [10/Aug/2022 17:27:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 17:27:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 17:27:38] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 17:27:38] "GET /_favicon.ico?v=2.6.1 HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 17:27:38] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Aug/2022 17:27:38] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Aug/2022 17:27:38] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 17:27:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 17:27:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 17:27:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 17:27:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 17:27:53] "POST /_dash-upda